In [2]:
#Import_Libraries
import pandas as pd
import numpy as np

In [4]:
#loading_dataset
df=pd.read_csv('wine.csv')
df

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


#Implementation_from_Scratch

In [7]:
def chi_square(df, attribute, target):
    attr_values=df[attribute].unique()
    target_values=df[target].unique()
    N=len(df)
    chi_square=0
    for value in attr_values:
        for value1 in target_values:
            observed_freq=len(df[attribute][df[attribute]==value][df[target]==value1])
            prob_attributes=len(df[attribute][df[attribute]==value])
            prob_target=len(df[target][df[target]==value1])
            expected_freq=(prob_attributes*prob_target)/N
            chi_square+=((observed_freq-expected_freq)**2) / (expected_freq)
    return chi_square / N

In [9]:
chi_square(df, 'Alcohol', 'Wine')

1.6450516896103615

#Using_above_fn._we_can_compute_the_chi-square_score_between_each_input_feature_and_choose_k_best_features

#Now_by_using_inBuilt_Fn.

In [16]:
X=df.iloc[:,:-1]
Y=df.iloc[:,-1]

In [18]:
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest


In [20]:
test=SelectKBest(score_func=chi2, k=4)
fit=test.fit(X, Y)

In [22]:
fit.scores_

array([ 46.36782609,   6.96021326,  65.34029499,   3.7494326 ,
        69.80332757, 268.7217397 ,  24.1088107 ,  71.36655174,
         5.40210573,  25.01632679, 134.65642138,   7.2367439 ,
        27.21591438])

In [24]:
X_new=test.fit_transform(X, Y)
X_new

array([[ 15.6     , 127.      ,   3.06    ,   5.64    ],
       [ 11.2     , 100.      ,   2.76    ,   4.38    ],
       [ 18.6     , 101.      ,   3.24    ,   5.68    ],
       [ 16.8     , 113.      ,   3.49    ,   7.8     ],
       [ 21.      , 118.      ,   2.69    ,   4.32    ],
       [ 15.2     , 112.      ,   3.39    ,   6.75    ],
       [ 14.6     ,  96.      ,   2.52    ,   5.25    ],
       [ 17.6     , 121.      ,   2.51    ,   5.05    ],
       [ 14.      ,  97.      ,   2.98    ,   5.2     ],
       [ 16.      ,  98.      ,   3.15    ,   7.22    ],
       [ 18.      , 105.      ,   3.32    ,   5.75    ],
       [ 16.8     ,  95.      ,   2.43    ,   5.      ],
       [ 16.      ,  89.      ,   2.76    ,   5.6     ],
       [ 11.4     ,  91.      ,   3.69    ,   5.4     ],
       [ 12.      , 102.      ,   3.64    ,   7.5     ],
       [ 17.2     , 112.      ,   2.91    ,   7.3     ],
       [ 20.      , 120.      ,   3.14    ,   6.2     ],
       [ 20.      , 115.      ,

#Method-2:Using_Information_Gain

In [27]:
def find_entropy(df):
    Class=df.keys()[-1]
    values=df[Class].unique()
    entropy=0
    for value in values:
        prob=len(df[df[Class]==value]) / len(df[Class])
        entropy+=-prob*np.log2(prob)
    return entropy

In [29]:
def find_information_entropy(df, attribute):
    Class=df.keys()[-1]
    labels=df[Class].unique()
    attr_values=df[attribute].unique()
    avg_entropy=0
    split_info=0
    for value1 in attr_values:
        entropy_subsample=0
        for value2 in labels:
            num=len(df[attribute][df[attribute]==value1][df[Class]==value2])
            den=len(df[attribute][df[attribute]==value1])
            prob=num / den
            entropy_subsample+=-(prob*np.log2(prob+1e-7))
        weight=den / len(df)
        avg_entropy+=weight*entropy_subsample
    return avg_entropy

In [31]:
def information_gain(df, attribute):
    return (find_entropy(df)-find_information_entropy(df, attribute))

In [33]:
information_gain(df, 'Alcohol')

6.036676217266828

In [37]:
from sklearn import datasets
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
test=SelectKBest(score_func=mutual_info_classif, k=2)
fit=test.fit(X, Y)
fit.scores_

python(31243) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


array([1.68050535, 0.16123938, 0.19415445, 0.        , 0.28988671,
       0.01340465, 0.11629187, 0.33601793, 0.09415608, 0.33424046,
       0.13662167, 0.27634784, 0.25287198])

In [39]:
X_new=test.fit_transform(X, Y)
X_new

array([[1.  , 2.29],
       [1.  , 1.28],
       [1.  , 2.81],
       [1.  , 2.18],
       [1.  , 1.82],
       [1.  , 1.97],
       [1.  , 1.98],
       [1.  , 1.25],
       [1.  , 1.98],
       [1.  , 1.85],
       [1.  , 2.38],
       [1.  , 1.57],
       [1.  , 1.81],
       [1.  , 2.81],
       [1.  , 2.96],
       [1.  , 1.46],
       [1.  , 1.97],
       [1.  , 1.72],
       [1.  , 1.86],
       [1.  , 1.66],
       [1.  , 2.1 ],
       [1.  , 1.98],
       [1.  , 1.69],
       [1.  , 1.46],
       [1.  , 1.66],
       [1.  , 1.92],
       [1.  , 1.45],
       [1.  , 1.35],
       [1.  , 1.76],
       [1.  , 1.98],
       [1.  , 2.38],
       [1.  , 1.95],
       [1.  , 1.97],
       [1.  , 1.35],
       [1.  , 1.54],
       [1.  , 1.86],
       [1.  , 1.36],
       [1.  , 1.44],
       [1.  , 1.37],
       [1.  , 2.08],
       [1.  , 2.34],
       [1.  , 1.48],
       [1.  , 1.7 ],
       [1.  , 1.66],
       [1.  , 2.03],
       [1.  , 1.25],
       [1.  , 2.19],
       [1.  ,

#Method-3:Removing_Features_with_low_variance

In [42]:
from sklearn.feature_selection import VarianceThreshold
sel=VarianceThreshold(threshold=(0.8*(1-0.8)))
X_new=sel.fit_transform(X)
X_new                      

array([[ 1.  , 14.23,  1.71, ...,  2.29,  5.64,  3.92],
       [ 1.  , 13.2 ,  1.78, ...,  1.28,  4.38,  3.4 ],
       [ 1.  , 13.16,  2.36, ...,  2.81,  5.68,  3.17],
       ...,
       [ 3.  , 13.27,  4.28, ...,  1.35, 10.2 ,  1.56],
       [ 3.  , 13.17,  2.59, ...,  1.46,  9.3 ,  1.62],
       [ 3.  , 14.13,  4.1 , ...,  1.35,  9.2 ,  1.6 ]])

#Method-4: Wrapper_Approach:using_Random_Forest_Classfifier

#1-By_training_a_Random_Forest_classfier_which_returns_feature_importances

In [49]:
from sklearn import datasets
iris=datasets.load_iris()

In [51]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
data = pd.DataFrame({'sepallength': iris.data[:, 0], 'sepalwidth': iris.data[:, 1],
                     'petallength': iris.data[:, 2], 'petalwidth': iris.data[:, 3],
                     'species': iris.target})

In [53]:
X, y=datasets.load_iris(return_X_y=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.30)

In [55]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [57]:
clf.feature_importances_

array([0.08799979, 0.03024963, 0.40238687, 0.47936371])

In [59]:
from sklearn.feature_selection import SelectFromModel

In [61]:
sel=SelectFromModel(RandomForestClassifier(n_estimators=100))
sel.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier())

In [65]:
sel.get_support()

array([False, False,  True,  True])